# 0. Project Set Up

In [83]:
# Import Python packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import json
import sys
import cachetools
from datetime import timedelta
import math

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window
from snowflake.snowpark.functions import col, to_date

# For display purposes, suppress FutureWarnings and UserWarnings from printing below the cells
import warnings
for warning_category in [FutureWarning, UserWarning]:
    warnings.filterwarnings('ignore', category=warning_category)

### 스노우 플레이크 연결 

In [86]:
from snowflake.snowpark.context import get_active_session 
from getpass import getpass

connection_parameters = {
    'account' : "mapbobj-vp47779",
    'user' : 'soojin',
    "password": 'Kimbusan51480501@',

    
    'warehouse' : 'HACKATHON',
    'database' : 'RAW_DATA',
    'schema': 'GRANDATA'
}

# 세션 수동 생성
session = Session.builder.configs(connection_parameters).create()

# 쿼리 태그 설정 (추적용)
session.query_tag = {
    'origin': 'digital_nomad_project', # 프로젝트 명
    'name' : 'remote_work_trend_analysis', # 작업 목적
    'version' : {'major' : 1, 'minor' : 0},
    'attributes' : {
        'source' : 'jupyter',
        'analyst' : 'soojin kim '
    }
}

# print the current role, warehouse, and db/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

role: "ACCOUNTADMIN" | WH: "HACKATHON" | DB.SCHEMA: "RAW_DATA"."GRANDATA"


In [87]:
# 재연결
from snowflake.snowpark import Session

connection_parameters = {
    "account": "mapbobj-vp47779",
    "user": "soojin",
    "password" : "Kimbusan51480501@",
    "warehouse": "HACKATHON",
    "database": "RAW_DATA",
    "schema": "GRANDATA",
    "authenticator": "snowflake"  # 브라우저 인증 방식
}

# 세션 재연결
session = Session.builder.configs(connection_parameters).create()



# 1. 데이터 확보 & 스키마 이해

In [90]:
# 테이블 로딩 
card_df = session.table("CARD_SALES_INFO")
float_df = session.table("FLOATING_POPULATION_INFO")
asset_df = session.table("ASSET_INCOME_INFO")
admin_df = session.table("M_SCCO_MST") # GEO 정보는 Panda로 export 후 geopandas로 처리 가능


# 일단 데이터 확인 
# Snowpark DataFrame → Pandas DataFrame
card_df_pd = card_df.to_pandas()
float_df_pd = float_df.to_pandas()
asset_df_pd = asset_df.to_pandas()
admin_df_pd = admin_df.to_pandas()


# 스키마(컬럼명, 데이터타입) 출력
print("📌 CARD_SALES_INFO schema:")
card_df.print_schema()
print("Total rows:", card_df.count())

print("\n📌 FLOATING_POPULATION_INFO schema:")
float_df.print_schema()
print("Total rows:", float_df.count())

print("\n📌 ASSET_INCOME_INFO schema:")
asset_df.print_schema()
print("Total rows:", asset_df.count())

print("\n📌 M_SCCO_MST schema:")
admin_df.print_schema()
print("Total rows:", admin_df.count())



📌 CARD_SALES_INFO schema:
root
 |-- "PROVINCE_CODE": StringType(2) (nullable = False)
 |-- "CITY_CODE": StringType(5) (nullable = False)
 |-- "DISTRICT_CODE": StringType(8) (nullable = False)
 |-- "STANDARD_YEAR_MONTH": StringType(6) (nullable = False)
 |-- "CARD_TYPE": StringType(1) (nullable = False)
 |-- "WEEKDAY_WEEKEND": StringType(1) (nullable = False)
 |-- "GENDER": StringType(1) (nullable = False)
 |-- "AGE_GROUP": StringType(2) (nullable = False)
 |-- "TIME_SLOT": StringType(3) (nullable = False)
 |-- "LIFESTYLE": StringType(3) (nullable = False)
 |-- "TOTAL_SALES": LongType() (nullable = True)
 |-- "FOOD_SALES": LongType() (nullable = True)
 |-- "COFFEE_SALES": LongType() (nullable = True)
 |-- "ENTERTAINMENT_SALES": LongType() (nullable = True)
 |-- "DEPARTMENT_STORE_SALES": LongType() (nullable = True)
 |-- "LARGE_DISCOUNT_STORE_SALES": LongType() (nullable = True)
 |-- "SMALL_RETAIL_STORE_SALES": LongType() (nullable = True)
 |-- "CLOTHING_ACCESSORIES_SALES": LongType() (n

In [92]:
# 일부 데이터 가져오기
card_sample = card_df.limit(5).to_pandas()
float_sample = float_df.limit(5).to_pandas()
asset_sample = asset_df.limit(5).to_pandas()
admin_sample = admin_df.limit(5).to_pandas()

# 출력
print("\n💳 CARD_SALES_INFO sample:")
print(card_sample)

print("\n👥 FLOATING_POPULATION_INFO sample:")
print(float_sample)

print("\n💰 ASSET_INCOME_INFO sample:")
print(asset_sample)

print("\n🗺️ M_SCCO_MST sample:")
print(admin_sample)



💳 CARD_SALES_INFO sample:
  PROVINCE_CODE CITY_CODE DISTRICT_CODE STANDARD_YEAR_MONTH CARD_TYPE  \
0            11     11140      11140129              202112         1   
1            11     11140      11140129              202112         1   
2            11     11140      11140129              202112         1   
3            11     11140      11140129              202112         1   
4            11     11140      11140129              202112         1   

  WEEKDAY_WEEKEND GENDER AGE_GROUP TIME_SLOT LIFESTYLE  ...  TRAVEL_COUNT  \
0               H      M        30       T15       L01  ...           0.0   
1               W      F        55       T21       L01  ...           0.0   
2               W      M        40       T12       L02  ...           0.0   
3               W      M        60       T15       L04  ...           0.0   
4               W      F        50       T12       L01  ...           0.0   

   BEAUTY_COUNT  HOME_LIFE_SERVICE_COUNT  EDUCATION_ACADEMY_COUNT  \
0 

In [93]:
print(card_df_pd.describe())

        TOTAL_SALES    FOOD_SALES  COFFEE_SALES  ENTERTAINMENT_SALES  \
count  3.742994e+06  3.742994e+06  3.742994e+06         3.742994e+06   
mean   1.493614e+08  3.979121e+06  1.041453e+06         8.040715e+04   
std    1.905431e+09  5.809149e+07  2.400028e+07         1.817873e+06   
min    5.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
25%    2.489410e+05  0.000000e+00  0.000000e+00         0.000000e+00   
50%    1.440312e+06  1.949780e+05  0.000000e+00         0.000000e+00   
75%    1.001267e+07  1.211793e+06  9.836500e+04         0.000000e+00   
max    6.528298e+11  1.488637e+10  7.385426e+09         5.892177e+08   

       DEPARTMENT_STORE_SALES  LARGE_DISCOUNT_STORE_SALES  \
count            3.742994e+06                3.742994e+06   
mean             4.105360e+06                1.811221e+06   
std              5.553114e+07                2.585855e+07   
min              0.000000e+00                0.000000e+00   
25%              0.000000e+00                0

# 2. 탄소 계수 (CF) 초기 테이블 만들기 
## 2-1. 소비탄소 지표 CSV 모으기 + 한화로 변경

In [120]:

import pandas as pd
from pathlib import Path
from forex_python.converter import CurrencyRates



In [148]:
RAW = Path("data/SupplyChainGHGEmissionFactors_v1.3.0_NAICS_CO2e_USD2022.csv")
OUT = Path("data/carbon_factor_supplychain_krw.csv")

# 1. csv 읽기 
df = pd.read_csv(RAW)

# 2 USD 단위 & 전체 GHG 열만 선택
mask = (
    (df["Unit"].str.contains("USD", na=False)) &
    (df["GHG"].str.contains("All", na=False))
)
df = df.loc[mask, ["2017 NAICS Title", "Supply Chain Emission Factors with Margins"]]
df = df.rename(columns={
        "2017 NAICS Title": "Category",
        "Supply Chain Emission Factors with Margins": "kgCO2e_per_USD"
     })

# 3. 환율 → 1,000원 기준 (상수 입력)
USD_KRW = 1400        # <─ 오늘자 환율 숫자만 바꿔 주세요
df["kgCO2e_per_1000KRW"] = df["kgCO2e_per_USD"] / (USD_KRW / 1000)

# 4. 저장
OUT.parent.mkdir(parents=True, exist_ok=True)
df[["Category", "kgCO2e_per_1000KRW"]].to_csv(OUT, index=False)
print("✅ CSV 저장 완료 →", OUT)
    

✅ CSV 저장 완료 → data\carbon_factor_supplychain_krw.csv


In [131]:
from pathlib import Path

RAW = Path(r"C:\Users\SSAFY\Desktop\HACKATHON_SnowflakeXStreamlit\data\SupplyChainGHGEmissionFactors_v1.3.0_NAICS_CO2e_USD2022.csv")
print("파일 존재?", RAW.exists())   # True여야 read_csv 가능
         # Desktop 폴더가 보이나?


파일 존재? True


## 2-2. 카드사 lifestyle 코드 <-> 탄소 카테고리 매핑표 작성

In [183]:
# 파일 경로 
CSV_FACTOR = "data/carbon_factor_supplychain_krw.csv"

# ▸ Snowflake 연결 (secrets.toml 쓰면 생략)
SNOW_CFG = {
    "account": "mapbobj-vp47779",
    "user": "soojin",
    "password" : "Kimbusan51480501@",
    "role": "ACCOUNTADMIN",
    'warehouse': "HACKATHON",
    'database' : "RAW_DATA",
    'schema'   : "GRANDATA"
}

# 1. 사용처 목록 자동 추출
usage_cols = [c for c in card_df.columns if c.endswith("_SALES")]
USAGES = [u.replace("_SALES", "") for u in usage_cols]
print(USAGES)   # ['FOOD','COFFEE', ... , 'E_COMMERCE']

['TOTAL', 'FOOD', 'COFFEE', 'ENTERTAINMENT', 'DEPARTMENT_STORE', 'LARGE_DISCOUNT_STORE', 'SMALL_RETAIL_STORE', 'CLOTHING_ACCESSORIES', 'SPORTS_CULTURE_LEISURE', 'ACCOMMODATION', 'TRAVEL', 'BEAUTY', 'HOME_LIFE_SERVICE', 'EDUCATION_ACADEMY', 'MEDICAL', 'ELECTRONICS_FURNITURE', 'CAR', 'CAR_SERVICE_SUPPLIES', 'GAS_STATION', 'E_COMMERCE']


In [185]:
# 2. 탄소계수 csv -> 사용처별 평균계수 자동 계산 

import pandas as pd, re

cf = pd.read_csv(CSV_FACTOR)

# 간단 키워드 사전 (필요 시 확장)
kw_dict = {
    "FOOD":       ["food", "restaurant", "catering"],
    "COFFEE":     ["coffee", "tea", "cafe"],
    "ENTERTAINMENT":["entertainment","cinema","theater","music"],
    "DEPARTMENT_STORE":["department store"],
    "LARGE_DISCOUNT_STORE":["hypermarket","discount", "warehouse club"],
    "SMALL_RETAIL_STORE":["convenience", "small retail"],
    "CLOTHING_ACCESSORIES":["clothing","apparel","footwear","jewelry"],
    "SPORTS_CULTURE_LEISURE":["sport","leisure","culture"],
    "ACCOMMODATION":["hotel","accommodation","lodging"],
    "TRAVEL":["travel","airline","tour","rail"],
    "BEAUTY":["beauty","cosmetics","hair","spa"],
    "HOME_LIFE_SERVICE":["home service","cleaning","repair"],
    "EDUCATION_ACADEMY":["education","academy","training"],
    "MEDICAL":["medical","hospital","clinic","pharmacy"],
    "ELECTRONICS_FURNITURE":["electronics","computer","furniture","appliance"],
    "CAR":["car sales","vehicle sales"],
    "CAR_SERVICE_SUPPLIES":["car service","auto parts","auto repair"],
    "GAS_STATION":["gas station","fuel","petrol"],
    "E_COMMERCE":["e-commerce","online retail","internet retail"],
}

rows=[]
for usage in USAGES:
    patt = re.compile("|".join(kw_dict.get(usage, [usage])), re.I)
    sub  = cf[cf["Category"].str.contains(patt, na=False)]
    if sub.empty:
        print("⚠️ 매핑 안된 사용처:", usage)
        factor = cf["kgCO2e_per_1000KRW"].mean()
    else:
        factor = sub["kgCO2e_per_1000KRW"].mean()      # ◆ 가중평균 넣을 곳
    rows.append({"USAGE": usage, "KGCO2E_PER_1000KRW": round(factor,4)})

usage_factor = pd.DataFrame(rows)
usage_factor.to_csv("usage_factor_table.csv", index=False)
usage_factor


⚠️ 매핑 안된 사용처: CAR
⚠️ 매핑 안된 사용처: E_COMMERCE


,USAGE,KGCO2E_PER_1000KRW
0,TOTAL,0.0600
1,FOOD,0.1907
2,COFFEE,0.2018
3,ENTERTAINMENT,0.0626
4,DEPARTMENT_STORE,0.1171
5,LARGE_DISCOUNT_STORE,0.1171
6,SMALL_RETAIL_STORE,0.1321
7,CLOTHING_ACCESSORIES,0.1058
8,SPORTS_CULTURE_LEISURE,0.3711
9,ACCOMMODATION,0.1087


In [193]:
# 3. snowflake: 테이블/뷰 생성 
from snowflake.snowpark import Session

session = Session.builder.configs(SNOW_CFG).create()

# 🔑 데이터베이스와 스키마 명시적으로 설정
session.sql("USE DATABASE MY_DB").collect()
session.sql("USE SCHEMA MY_SCHEMA").collect()


# 3-1 테이블 업로드
session.sql("""
    CREATE OR REPLACE TABLE USAGE_FACTOR_TBL (
        USAGE STRING,
        KGCO2E_PER_1000KRW FLOAT
    )
""").collect()


session.write_pandas(usage_factor, "USAGE_FACTOR_TBL", overwrite=True)


session.table("USAGE_FACTOR_TBL").limit(5).to_pandas()


,USAGE,KGCO2E_PER_1000KRW
0,TOTAL,0.0600
1,FOOD,0.1907
2,COFFEE,0.2018
3,ENTERTAINMENT,0.0626
4,DEPARTMENT_STORE,0.1171


In [211]:
all_columns = [
    'PROVINCE_CODE', 'CITY_CODE', 'DISTRICT_CODE', 'STANDARD_YEAR_MONTH',
    'CARD_TYPE', 'WEEKDAY_WEEKEND', 'GENDER', 'AGE_GROUP', 'TIME_SLOT', 'LIFESTYLE',
    'TOTAL_SALES', 'FOOD_SALES', 'COFFEE_SALES', 'ENTERTAINMENT_SALES',
    'DEPARTMENT_STORE_SALES', 'LARGE_DISCOUNT_STORE_SALES', 'SMALL_RETAIL_STORE_SALES',
    'CLOTHING_ACCESSORIES_SALES', 'SPORTS_CULTURE_LEISURE_SALES', 'ACCOMMODATION_SALES',
    'TRAVEL_SALES', 'BEAUTY_SALES', 'HOME_LIFE_SERVICE_SALES', 'EDUCATION_ACADEMY_SALES',
    'MEDICAL_SALES', 'ELECTRONICS_FURNITURE_SALES', 'CAR_SALES', 'CAR_SERVICE_SUPPLIES_SALES',
    'GAS_STATION_SALES', 'E_COMMERCE_SALES', 'TOTAL_COUNT', 'FOOD_COUNT', 'COFFEE_COUNT',
    'ENTERTAINMENT_COUNT', 'DEPARTMENT_STORE_COUNT', 'LARGE_DISCOUNT_STORE_COUNT',
    'SMALL_RETAIL_STORE_COUNT', 'CLOTHING_ACCESSORIES_COUNT', 'SPORTS_CULTURE_LEISURE_COUNT',
    'ACCOMMODATION_COUNT', 'TRAVEL_COUNT', 'BEAUTY_COUNT', 'HOME_LIFE_SERVICE_COUNT',
    'EDUCATION_ACADEMY_COUNT', 'MEDICAL_COUNT', 'ELECTRONICS_FURNITURE_COUNT',
    'CAR_SALES_COUNT', 'CAR_SERVICE_SUPPLIES_COUNT', 'GAS_STATION_COUNT', 'E_COMMERCE_COUNT'
]

# USAGES 리스트: *_SALES 중 TOTAL_SALES 제외
USAGES = [col.replace('_SALES', '') for col in all_columns if col.endswith('_SALES') and col != 'TOTAL_SALES']


In [225]:
unp_cols = ", ".join([f"{usage}_SALES" for usage in USAGES])

create_view = f"""
CREATE OR REPLACE VIEW CARD_CO2E_VW AS
WITH unp AS (
  SELECT 
    PROVINCE_CODE,
    CITY_CODE,
    DISTRICT_CODE,
    STANDARD_YEAR_MONTH,
    CARD_TYPE,
    WEEKDAY_WEEKEND,
    GENDER,
    AGE_GROUP,
    TIME_SLOT,
    LIFESTYLE,
    REPLACE(usage, '_SALES', '') AS usage_clean,
    value AS sales_amt
  FROM RAW_DATA.GRANDATA.CARD_SALES_INFO
  UNPIVOT(value FOR usage IN ({unp_cols}))
)
SELECT 
    unp.*,
    f.KGCO2E_PER_1000KRW,
    (sales_amt / 1000) * f.KGCO2E_PER_1000KRW AS CO2E_KG
FROM unp
LEFT JOIN USAGE_FACTOR_TBL f
  ON upper(unp.usage_clean) = upper(f.USAGE);
"""

session.sql(create_view).collect()
print("✅ CARD_CO2E_VW 뷰 재생성 완료")


✅ CARD_CO2E_VW 뷰 재생성 완료


In [227]:
session.table("CARD_CO2E_VW").limit(5).to_pandas()


,PROVINCE_CODE,CITY_CODE,DISTRICT_CODE,STANDARD_YEAR_MONTH,CARD_TYPE,WEEKDAY_WEEKEND,GENDER,AGE_GROUP,TIME_SLOT,LIFESTYLE,USAGE_CLEAN,SALES_AMT,KGCO2E_PER_1000KRW,CO2E_KG
0,11,11140,11140129,202112,1,H,M,30,T15,L01,FOOD,3027922,0.1907,577.424725
1,11,11140,11140129,202112,1,H,M,30,T15,L01,COFFEE,862684,0.2018,174.089631
2,11,11140,11140129,202112,1,H,M,30,T15,L01,ENTERTAINMENT,0,0.0626,0.000000
3,11,11140,11140129,202112,1,H,M,30,T15,L01,DEPARTMENT_STORE,0,0.1171,0.000000
4,11,11140,11140129,202112,1,H,M,30,T15,L01,LARGE_DISCOUNT_STORE,0,0.1171,0.000000


In [229]:
session.table("USAGE_FACTOR_TBL").to_pandas()


,USAGE,KGCO2E_PER_1000KRW
0,TOTAL,0.0600
1,FOOD,0.1907
2,COFFEE,0.2018
3,ENTERTAINMENT,0.0626
4,DEPARTMENT_STORE,0.1171
5,LARGE_DISCOUNT_STORE,0.1171
6,SMALL_RETAIL_STORE,0.1321
7,CLOTHING_ACCESSORIES,0.1058
8,SPORTS_CULTURE_LEISURE,0.3711
9,ACCOMMODATION,0.1087


In [236]:
session.table("CARD_CO2E_VW").limit(10).to_pandas()


,PROVINCE_CODE,CITY_CODE,DISTRICT_CODE,STANDARD_YEAR_MONTH,CARD_TYPE,WEEKDAY_WEEKEND,GENDER,AGE_GROUP,TIME_SLOT,LIFESTYLE,USAGE_CLEAN,SALES_AMT,KGCO2E_PER_1000KRW,CO2E_KG
0,11,11140,11140112,202206,1,H,M,35,T12,L04,FOOD,596328,0.1907,113.719750
1,11,11140,11140112,202206,1,H,M,35,T12,L04,COFFEE,27795,0.2018,5.609031
2,11,11140,11140112,202206,1,H,M,35,T12,L04,ENTERTAINMENT,0,0.0626,0.000000
3,11,11140,11140112,202206,1,H,M,35,T12,L04,DEPARTMENT_STORE,0,0.1171,0.000000
4,11,11140,11140112,202206,1,H,M,35,T12,L04,LARGE_DISCOUNT_STORE,0,0.1171,0.000000
5,11,11140,11140112,202206,1,H,M,35,T12,L04,SMALL_RETAIL_STORE,10107,0.1321,1.335135
6,11,11140,11140112,202206,1,H,M,35,T12,L04,CLOTHING_ACCESSORIES,126341,0.1058,13.366878
7,11,11140,11140112,202206,1,H,M,35,T12,L04,SPORTS_CULTURE_LEISURE,0,0.3711,0.000000
8,11,11140,11140112,202206,1,H,M,35,T12,L04,ACCOMMODATION,0,0.1087,0.000000
9,11,11140,11140112,202206,1,H,M,35,T12,L04,TRAVEL,0,0.1715,0.000000


In [238]:
# 월, 사용처별 CO2 집계 확인
# 월별·사용처별 CO₂e, 매출도 같이 보고 싶다면
session.sql("""
SELECT  STANDARD_YEAR_MONTH,
        usage_clean            AS USAGE,
        SUM(sales_amt)         AS TOTAL_SALES,
        SUM(CO2E_KG)           AS TOTAL_CO2E_KG
FROM    CARD_CO2E_VW
GROUP BY 1,2
ORDER BY 1 DESC, 2
LIMIT 20
""").show()


--------------------------------------------------------------------------------------
|"STANDARD_YEAR_MONTH"  |"USAGE"                |"TOTAL_SALES"  |"TOTAL_CO2E_KG"     |
--------------------------------------------------------------------------------------
|202312                 |ACCOMMODATION          |82114660653    |8925863.612981101   |
|202312                 |BEAUTY                 |34279869869    |4236991.915808401   |
|202312                 |CAR                    |720534236466   |145115595.2242524   |
|202312                 |CAR_SERVICE_SUPPLIES   |33522550110    |1773342.9008189999  |
|202312                 |CLOTHING_ACCESSORIES   |52080090221    |5510073.5453818     |
|202312                 |COFFEE                 |127529325595   |25735417.905071     |
|202312                 |DEPARTMENT_STORE       |552262545597   |64669944.08940871   |
|202312                 |EDUCATION_ACADEMY      |152106449347   |11727407.244653698  |
|202312                 |ELECTRONICS_FURNIT

In [242]:
import streamlit as st, plotly.express as px
from snowflake.snowpark import Session
import pandas as pd

st.set_page_config(page_title="사용처별 탄소 배출", layout="wide")

# 1) 세션
sess = Session.builder.configs(st.secrets).create()

# 2) 월 선택 ▸ DISTINCT 가져오기
months = sess.sql("""
    SELECT DISTINCT STANDARD_YEAR_MONTH
    FROM CARD_CO2E_VW
    ORDER BY 1 DESC
""").to_pandas()["STANDARD_YEAR_MONTH"]

sel_month = st.sidebar.selectbox("조회 월", months)

# 3) 데이터 가져오기
df = sess.sql(f"""
    SELECT  usage_clean AS usage,
            SUM(sales_amt)  AS total_sales,
            SUM(CO2E_KG)    AS co2e_kg
    FROM CARD_CO2E_VW
    WHERE STANDARD_YEAR_MONTH = '{sel_month}'
    GROUP BY usage_clean
    ORDER BY co2e_kg DESC
""").to_pandas()

# 4) 지표 + 그래프
st.metric("총 CO₂ 배출(kg)", f"{df['co2e_kg'].sum():,.0f}")

fig = px.bar(df, x="usage", y="co2e_kg",
             title=f"{sel_month} 사용처별 CO₂ 배출",
             labels={"co2e_kg":"kg CO₂e", "usage":"사용처"})
st.plotly_chart(fig, use_container_width=True)

# 5) Raw table
with st.expander("원본 데이터 보기"):
    st.dataframe(df)


FileNotFoundError: No secrets files found. Valid paths for a secrets.toml file are: C:\Users\SSAFY\.streamlit\secrets.toml, C:\Users\SSAFY\Desktop\HACKATHON_SnowflakeXStreamlit\.streamlit\secrets.toml

In [1]:
# ! pip install streamlit snowflake-connector-python snowflake-snowpark-python plotly pandas